API Documentation: https://ygoprodeck.com/api-guide/

In [1]:
#import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import numpy as np

In [2]:
#Set banlist URL to a variable
url = 'https://www.yugioh-card.com/en/limited/list_2024-01-01/'

#Make a connection to the URL
response = requests.get(url)

In [3]:
#If the connection is successful, extract the HTML of the url.
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
#Create lists to store scraped information
cardtype = []
cardname = []
advformat = []
tradformat = []
remarks = []

#Variable to store all of the tr tags from the HTML content.
limitcards = soup.find_all('tr')

#For loop that goes through each td element from each tr element.
for td in limitcards:
    #Gather all td elements from the current tr element.
    td_list = td.find_all('td')
    #Since the elements are stored in a list, we can append information to the their respective list based on the position from the td_list.
    #Each append is done by only extracting the text, the relevant info.
    cardtype.append(td_list[0].get_text())
    cardname.append(td_list[1].get_text())
    advformat.append(td_list[2].get_text())
    #Attempt to extract from the remarks index. Otherwise, move on to except.
    #Needed since the index position shifts during the extraction process.
    try:
        remarks.append(td_list[4].get_text())
        tradformat.append(td_list[3].get_text())
    except:
        tradformat.append(td_list[2].get_text())
        remarks.append(td_list[3].get_text())
    

In [5]:
#Create dictionary for DF1
DF1_dict = {
    'Card Type' : cardtype,
    'Card Name' : cardname,
    'Advanced Format' : advformat,
    'Tradional Format' : tradformat,
    'Remarks' : remarks
}

In [6]:
#Create data frame using DF1_dict
DF1 = pd.DataFrame(DF1_dict)
#Delete first row since it is the column names
DF1.drop(axis=0, index=0, inplace=True)
#Delete Card Name since column names repeat.
DF1 = DF1.drop(DF1[DF1['Card Name'] == 'Card Name'].index)
#Reset index to avoid conflicts
DF1 = DF1.reset_index()
#Delete old index column
DF1 = DF1.drop(columns=['index'])
DF1

,Card Type,Card Name,Advanced Format,Tradional Format,Remarks
0,Monster/Effect,AGIDO THE ANCIENT SENTINEL,Forbidden,Limited,Was Limited
1,Monster/Effect,ARCHNEMESES PROTOS,Forbidden,Limited,
2,Monster/Effect,ARTIFACT SCYTHE,Forbidden,Limited,
3,Monster/Effect,BARRIER STATUE OF THE STORMWINDS,Forbidden,Limited,
4,Monster/Effect,BLACKWING – GOFU THE VAGUE SHADOW,Forbidden,Limited,
...,...,...,...,...,...
196,Spell,MIND CONTROL,No Longer On List,No Longer On List,Was Limited
197,Spell,POT OF DESIRES,No Longer On List,No Longer On List,Was Semi-Limited
198,Spell,SPELLBOOK OF JUDGMENT,No Longer On List,No Longer On List,Was Limited
199,Spell,SPRIGHT STARTER,No Longer On List,No Longer On List,Was Semi-Limited


In [7]:
#Make a new list of card names based on DF1
cardname_df = DF1['Card Name'].values.tolist()
#Make a new list to have proper styling for card names.
cardname_final = []
#Replace the 4 spaces in the card name with a single space.
#Also replace – with -. 
#Done to avoid conflict with URLs.
for i in cardname_df:
    i = i.replace('    ', ' ').replace('–', '-')
    cardname_final.append(i)

In [8]:
#Create lists to store data for DF2
cardid = []
cardchar = [] #refers to race from json
cardarch = []
numsetsreleased = []
#There 3 lists are the most reliable sources for buying cards.
tcgprice = []
ebayprice = []
amazonprice = []

#For loop that makes an API request for each card name gathered.
for name in cardname_final:
    #Rename the API URL for each card name in the list.
    api_url = f'https://db.ygoprodeck.com/api/v7/cardinfo.php?name={name}'
    response = requests.get(api_url)
    
    #If we get a successful connection, begin extracting data.
    if response.status_code == 200:
        json_content = (response.content)
        #Getting json from webpage through loads.
        #Help from https://stackoverflow.com/questions/12965203/how-to-get-json-from-webpage-into-python-script
        scraped_json = json.loads(json_content)
        
        #Create varaible to avoid repeating ['data'][0]
        json_data = scraped_json['data'][0]
        
        #Append each list with their respective values from the JSON
        cardid.append(json_data['id'])
        cardchar.append(json_data['race'])
        numsetsreleased.append(len(json_data['card_sets']))
        tcgprice.append(json_data['card_prices'][0]['tcgplayer_price'])
        ebayprice.append(json_data['card_prices'][0]['ebay_price'])
        amazonprice.append(json_data['card_prices'][0]['amazon_price'])
        #Check if there is an archetype associated with the card
        #If there is none, use a NaN value.
        try:
            cardarch.append(json_data['archetype'])
        except:
            cardarch.append(np.NaN)

In [9]:
#Create dictionary for DF2
DF2_dict = {
    'Card ID' : cardid,
    'Card Detail' : cardchar,
    'Card Archetype' : cardarch,
    '# of Sets Released In' : numsetsreleased,
    'TCGPlayer Price Low' : tcgprice,
    'eBay Price Low': ebayprice,
    'Amazon Price Low': amazonprice
}

In [10]:
#Create DF2 data frame
DF2 = pd.DataFrame(DF2_dict)
DF2

,Card ID,Card Detail,Card Archetype,# of Sets Released In,TCGPlayer Price Low,eBay Price Low,Amazon Price Low
0,62320425,Fairy,Exchange of the Spirit,1,0.67,1.99,7.44
1,6728559,Wyrm,Nemeses,1,5.83,8.06,0.50
2,20292186,Fairy,Artifact,5,0.13,1.79,2.14
3,73356503,Winged Beast,NaN,2,0.35,2.40,3.42
4,9929398,Winged Beast,Blackwing,3,0.17,3.95,0.25
...,...,...,...,...,...,...,...
196,37520316,Normal,NaN,9,0.24,1.99,0.49
197,35261759,Normal,Greed,11,0.26,2.25,5.94
198,46448938,Quick-Play,Spellbook,4,0.07,4.99,8.17
199,15443125,Quick-Play,Spright,1,4.60,9.19,17.99


In [11]:
#Join the two dataframes horizontally by stating axis=1
DF3 = pd.concat([DF1, DF2], axis=1)
DF3

,Card Type,Card Name,Advanced Format,Tradional Format,Remarks,Card ID,Card Detail,Card Archetype,# of Sets Released In,TCGPlayer Price Low,eBay Price Low,Amazon Price Low
0,Monster/Effect,AGIDO THE ANCIENT SENTINEL,Forbidden,Limited,Was Limited,62320425,Fairy,Exchange of the Spirit,1,0.67,1.99,7.44
1,Monster/Effect,ARCHNEMESES PROTOS,Forbidden,Limited,,6728559,Wyrm,Nemeses,1,5.83,8.06,0.50
2,Monster/Effect,ARTIFACT SCYTHE,Forbidden,Limited,,20292186,Fairy,Artifact,5,0.13,1.79,2.14
3,Monster/Effect,BARRIER STATUE OF THE STORMWINDS,Forbidden,Limited,,73356503,Winged Beast,NaN,2,0.35,2.40,3.42
4,Monster/Effect,BLACKWING – GOFU THE VAGUE SHADOW,Forbidden,Limited,,9929398,Winged Beast,Blackwing,3,0.17,3.95,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
196,Spell,MIND CONTROL,No Longer On List,No Longer On List,Was Limited,37520316,Normal,NaN,9,0.24,1.99,0.49
197,Spell,POT OF DESIRES,No Longer On List,No Longer On List,Was Semi-Limited,35261759,Normal,Greed,11,0.26,2.25,5.94
198,Spell,SPELLBOOK OF JUDGMENT,No Longer On List,No Longer On List,Was Limited,46448938,Quick-Play,Spellbook,4,0.07,4.99,8.17
199,Spell,SPRIGHT STARTER,No Longer On List,No Longer On List,Was Semi-Limited,15443125,Quick-Play,Spright,1,4.60,9.19,17.99


In [12]:
#Most values were returned as objects. We need to convert them to their proper type.
#Infer objects did not work. Borrowed code from: https://medium.com/@dataproducts/pandas-detect-mixed-data-types-and-fix-it-7f0bd3b8d071
#For loop that goes through every column and tries to convert the type to a number format.
#If it cannot, move on to the next column.
for column in DF3.columns:
    DF3[column] = DF3[column].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))
#Convert remaining objects into strings using the best possible dtype
#Found from: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.convert_dtypes.html
DF3 = DF3.convert_dtypes()
#Check if column types were successfully transformed.
DF3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Card Type              201 non-null    string 
 1   Card Name              201 non-null    string 
 2   Advanced Format        201 non-null    string 
 3   Tradional Format       201 non-null    string 
 4   Remarks                201 non-null    string 
 5   Card ID                201 non-null    Int64  
 6   Card Detail            201 non-null    string 
 7   Card Archetype         100 non-null    string 
 8   # of Sets Released In  201 non-null    Int64  
 9   TCGPlayer Price Low    201 non-null    Float64
 10  eBay Price Low         201 non-null    Float64
 11  Amazon Price Low       201 non-null    Float64
dtypes: Float64(3), Int64(2), string(7)
memory usage: 20.0 KB


In [13]:
#Remove Card ID column since it is unnecessary for description statistics
DF3.drop(columns = 'Card ID', inplace=True)

#Display description statistics from DF3 dataframe
DF3.describe()

,# of Sets Released In,TCGPlayer Price Low,eBay Price Low,Amazon Price Low
count,201.0,201.0,201.0,201.0
mean,5.278607,1.973682,8.493333,6.550995
std,5.953318,4.669567,29.236396,12.152597
min,1.0,0.04,0.0,0.0
25%,2.0,0.24,1.75,0.96
50%,3.0,0.52,2.95,2.14
75%,6.0,1.46,5.95,6.18
max,38.0,40.13,289.95,89.69


In [14]:
#Convert DF3 dataframe into a csv file
DF3.to_csv('HW3_Q1_DF3_RobertNeagu.csv')